In [ ]:
import os

# Importing modules to run an agentic tool
import io
import contextlib
import traceback
import textwrap

# Importing modules required for prompt parsing, vector creation, tool/LLM invocation etc.
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

#Importing the caching modules
from langchain_core.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

# Importing in-memory storage (stateful) modules
from langgraph.checkpoint.memory import MemorySaver


######################################################################################################################################################
# Step 1: Set OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is not set")

# Step 1 sanity check (print last 4 chars of the API key)
print("Using key ending with:", api_key[-4:])
# Define the tool for context retrieval

# Step 1.1: Enable persistent caching for LLM responses
set_llm_cache(SQLiteCache("llm_cache.db"))

# Step 1.1 anity check (print cache name)
print("Using SQLite LLM cache at llm_cache.db")

@tool
#Retrieval and Generation
def retrieve_context(query: str):
    """Search for relevant coding documents."""
    urls = [
        "http://www.w3schools.com/",
        "https://docs.python.org/3/tutorial/index.html",
        "https://realpython.com/python-basics/",
        "https://www.learnpython.org/"
    ]

    # Load documents
    loader = UnstructuredURLLoader(urls=urls)
    docs = loader.load()

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=50,
    )
    doc_splits = text_splitter.split_documents(docs)

    # Create VectorStore (FAISS instead of Chroma)
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(
        documents=doc_splits,
        embedding=embeddings,
    )

    retriever = vectorstore.as_retriever()
    results = retriever.invoke(query)

    return "\n".join([doc.page_content for doc in results])

@tool
# A Python code runner tool to provide output and explanation after code execution
def run_python(code: str) -> str:
    """Execute a small Python snippet and return its output or error.

    Use this when you need to see the real behavior of a code example,
    such as what it prints or whether it raises an error.
    """
    # Normalize indentation (helps when users paste indented code)
    cleaned_code = textwrap.dedent(code).strip()

    # Buffer to capture standard output
    buf = io.StringIO()

    # Isolated namespace for this execution
    local_vars = {}

    try:
        # Capture stdout while running the code
        with contextlib.redirect_stdout(buf):
            exec(cleaned_code, {}, local_vars)

        stdout = buf.getvalue().strip()

        # Remove internal keys like __builtins__
        safe_vars = {
            k: repr(v) for k, v in local_vars.items()
            if k != "__builtins__"
        }

        result_lines = []
        result_lines.append("Code executed successfully.")

        # Show stdout if any
        if stdout:
            result_lines.append("\n[stdout]")
            result_lines.append(stdout)

        # Show resulting variables if any
        if safe_vars:
            result_lines.append("\n[variables]")
            for name, value_repr in safe_vars.items():
                result_lines.append(f"{name} = {value_repr}")

        return "\n".join(result_lines) or "Code executed (no output, no variables)."

    except Exception:
        stdout = buf.getvalue().strip()
        tb = traceback.format_exc()

        result_lines = []
        result_lines.append("Code execution failed.")

        if stdout:
            result_lines.append("\n[partial stdout before error]")
            result_lines.append(stdout)

        result_lines.append("\n[traceback]")
        result_lines.append(tb)

        return "\n".join(result_lines)


tools = [retrieve_context, run_python]
tool_node = ToolNode(tools)

# OpenAI LLM model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools(tools)

# Function to decide whether to continue or stop the workflow
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, go to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, finish the workflow
    return END

# Function that invokes the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}  # Returns as a list to add to the state

# Define the workflow with LangGraph
workflow = StateGraph(MessagesState)

# Add nodes to the graph
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Connect nodes
workflow.add_edge(START, "agent")  # Initial entry
workflow.add_conditional_edges("agent", should_continue)  # Decision after the "agent" node
workflow.add_edge("tools", "agent")  # Cycle between tools and agent

# Configure memory to persist the state
checkpointer = MemorySaver()

# Compile the graph into a LangChain Runnable application
app = workflow.compile(checkpointer=checkpointer)

# Execute the workflow || invoking the app for a specific task

# first define a logical conversation/session ID
thread_config = {"configurable": {"thread_id": "python-thread-1"}} 

# Show response to first question
final_state_1 = app.invoke(
    {"messages": [HumanMessage(content="Explain what is object oriented programming")]},
    config=thread_config
)

# Show response to first question
print(final_state_1["messages"][-1].content+"\n\n\n")

final_state_2 = app.invoke(
    {"messages": [HumanMessage(content="Explain how to put a heading on a webpage")]},
    config=thread_config
)

# Show response to second question
print(final_state_2["messages"][-1].content+"\n\n\n")

#Python code evaluator tool

# Ask user for a code snippet
user_code = input("Enter Python code to evaluate:\n\n")

# Wrap the code in a natural-language request so the agent knows what to do
user_prompt = f"Please run this code and explain the output or error:\n\n{user_code}"

#Invoke the python code evaluator tool
final_state_4 = app.invoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config=thread_config
)

# Print final assistant message
print("\n--- Agent Response ---\n")
print(final_state_4["messages"][-1].content)

Using key ending with: p9MA
Using SQLite LLM cache at llm_cache.db
Object-oriented programming (OOP) is a programming paradigm that organizes software design around objects rather than actions and data rather than logic. In OOP, objects are instances of classes, which define the structure and behavior of the objects. 

Key concepts of object-oriented programming include:

1. **Classes**: Classes are blueprints for creating objects. They define the properties (attributes) and behaviors (methods) that objects of that class will have.

2. **Objects**: Objects are instances of classes. They represent real-world entities and encapsulate data (attributes) and behavior (methods).

3. **Encapsulation**: Encapsulation is the bundling of data (attributes) and methods that operate on the data into a single unit (class). It helps in data hiding and abstraction.

4. **Inheritance**: Inheritance allows a class (subclass) to inherit properties and behavior from another class (superclass). It promotes